In [87]:
# Load Libraries
import pandas as pd

In [88]:
# Viếng thăm

# Define the file path
file_path = 'Dataset/dms.visit.partner_visit.xlsx'

# Load the Excel file
df_visit = pd.read_excel(file_path)

# Group by 'Nhân viên viếng thăm' and count the number of visits
visit_counts = df_visit.groupby('Nhân viên viếng thăm').size().reset_index(name='Tổng số lượt đi')

# Display the result
visit_counts


,Nhân viên viếng thăm,Tổng số lượt đi
0,MT- MEKONG1,33
1,MT- MIENDONG,69
2,MT- MIENTRUNG,47
3,MT-QLKDMN,22
4,MT-QLKDMN3,1
5,MTHCM-KV1,85
6,MTHCM-KV2,92
7,MTHCM-KV3,105
8,MTHCM-KV5,102
9,MTHCM-KV6,75


In [89]:
# Độ phủ

# 
# Define the file paths
file_path_fill = 'Dataset/dms.visit.partner_fill.xlsx'

file_path_customer = 'Dataset/customer.xlsx'  # Update this to the correct path for df_customer

df_fill = pd.read_excel(file_path_fill)
# CUSTOMER -> MERGED
# Khách hàng
df_customer=pd.read_excel(file_path_customer,sheet_name='Data khách hàng')

# Merge df_fill with df_customer to get 'Nhân viên KD' and 'Loại hình'
df_merged = pd.merge(df_fill, df_customer[['Mã', 'Nhân viên KD','Hệ thống' ,'Loại hình']], left_on='Khách hàng/Mã', right_on='Mã', how='right')


# Bỏ quản lý
list_quan_ly = ['MT-QLKDMN','MT-QLKDMN3']

df_fillmerged=df_merged[-df_merged['Nhân viên viếng thăm'].isin(list_quan_ly)].rename(columns={
    'Chi tiết kiểm tồn/Sản phẩm/Mã nội bộ': 'Mã sản phẩm',
    'Khách hàng/Mã': 'Mã khách hàng',
    'Khách hàng' : 'Mã khách hàng ERP',
    'Khách hàng/Tên':'Tên khách hàng',
    'Chi tiết kiểm tồn/Sản phẩm/Tên': 'Tên sản phẩm',
    'Chi tiết kiểm tồn/Số lượng chẵn': 'Tồn',
    'Chi tiết kiểm tồn/Sản phẩm/Đơn vị tính':'Đơn vị tính'
})


# Convert 'Đăng nhập' to datetime format if it isn't already
df_fillmerged['Đăng nhập'] = pd.to_datetime(df_fillmerged['Đăng nhập'], errors='coerce')

# Create a new column 'Date_Signin' by extracting the date from 'Đăng nhập'
df_fillmerged['Date_Signin'] = df_fillmerged['Đăng nhập'].dt.date



# Group by 'Mã khách hàng', 'Mã sản phẩm', 'Date_Signin' and calculate the sum of 'Tồn'
df_grouped = df_fillmerged.groupby(['Mã khách hàng', 'Mã sản phẩm', 'Date_Signin']).agg({'Tồn': 'sum'}).reset_index()

# Add 'Check_Show' and 'Product_Exist' columns based on the 'Tồn' value
df_grouped['Check_Show'] = df_grouped['Tồn'].apply(lambda x: 1 if x > 0 else 0)

# Add 'Product_Exist' column
df_grouped['Product_Exist'] = df_grouped['Mã sản phẩm'].notna().astype(int)

df_fill_store = df_grouped.merge(df_fillmerged[['Mã khách hàng', 'Mã sản phẩm', 'Nhân viên KD', 'Loại hình','Hệ thống']], on=['Mã khách hàng', 'Mã sản phẩm'], how='left').drop_duplicates()

# Group by 'Nhân viên KD', 'Loại hình' and calculate the sum of 'Check_Show' and 'Product_Exist'
df_pivot = df_fill_store.groupby(['Nhân viên KD', 'Loại hình']).agg({'Check_Show': 'sum', 'Product_Exist': 'sum'}).reset_index()

# Calculate 'Fill_Store_percent' as Sum(Check_Show) / Sum(Product_Exist) * 100
df_pivot['Fill_Store_percent'] = (df_pivot['Check_Show'] / df_pivot['Product_Exist'])

pivot_fill_store = df_pivot.pivot_table(index='Nhân viên KD', columns='Loại hình', values='Fill_Store_percent').reset_index()


# Calculate grand totals for each column
pivot_fill_store['Grand Total'] = pivot_fill_store.mean(axis=1, numeric_only=True)

# Calculate the overall grand total as the mean of the grand total column
grand_total = pivot_fill_store['Grand Total'].mean()

# Add the grand total row
pivot_fill_store.loc['Grand Total'] = pivot_fill_store.mean(numeric_only=True)
pivot_fill_store.at['Grand Total', 'Nhân viên KD'] = 'Grand Total'

# Calculate and add the overall grand total in the grand total row
pivot_fill_store.at['Grand Total', 'Grand Total'] = grand_total

pivot_fill_store

Loại hình,Nhân viên KD,Siêu Thị,Win+,Đại Siêu Thị,Grand Total
0,MT- MEKONG,NaN,1.000000,0.405784,0.702892
1,MT- MIENDONG,0.731225,1.000000,0.997699,0.909641
2,MTHCM-KV1,0.444654,1.000000,0.585831,0.676828
3,MTHCM-KV2,0.259409,1.000000,0.587407,0.615605
4,MTHCM-KV3,0.344159,0.857143,0.718182,0.639828
5,MTHCM-KV5,0.286045,0.555556,0.572541,0.471380
6,MTHCM-KV6,0.300582,1.000000,0.542662,0.614415
7,MTHCM-KV7,0.368056,0.805556,0.574394,0.582669
Grand Total,Grand Total,0.390590,0.902282,0.623062,0.651657


In [ ]:
# Export dataframes to multiple sheets in an Excel file
output_file = 'Dataset/output.xlsx'  # Define the output file path

with pd.ExcelWriter(output_file) as writer:
    df_fillmerged.to_excel(writer, sheet_name='df_fill', index=False)
    df_fill_store.to_excel(writer, sheet_name='df_fill_store', index=False)
    df_pivot.to_excel(writer, sheet_name='df_pivot', index=False)
    pivot_fill_store.to_excel(writer, sheet_name='pivot_fill_store', index=False)

print(f"Dataframes exported successfully to {output_file}")